# Data Exploration

In [2]:
import pandas as pd
import matplotlib as plt
import seaborn as sns

## a)

In [ ]:
df = pd.read_csv("C:/Users/Postb/Documents/GitHub/IDS/data/insurance_orig.csv")
rows, cols = df.shape
print(f"Rows: {rows}")
print(f"Columns: {cols}")


FileNotFoundError: [Errno 2] No such file or directory: 'data/insurance_orig.csv'

## b)

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
from IPython.display import Image, display

data_path = Path('data/insurance_orig.csv')
if not data_path.exists():
    data_path = Path('..') / 'data' / 'insurance_orig.csv'

if 'df' not in globals():
    df = pd.read_csv(data_path)
summary_stats = df.describe()
display(summary_stats)

artifacts_dir = Path('artifacts')
artifacts_dir.mkdir(parents=True, exist_ok=True)
image_path = artifacts_dir / 'q1_describe_table.png'

fig, ax = plt.subplots(figsize=(10, 2.75))
ax.axis('off')
table = ax.table(
    cellText=summary_stats.round(2).values,
    rowLabels=summary_stats.index,
    colLabels=summary_stats.columns,
    loc='center'
)
table.auto_set_font_size(False)
table.set_fontsize(8)
table.scale(1, 1.3)
for cell_obj in table.get_celld().values():
    cell_obj.set_edgecolor('#cccccc')
fig.tight_layout()
fig.savefig(image_path, dpi=200, bbox_inches='tight')
plt.close(fig)

display(Image(filename=str(image_path)))
print(f"Saved describe() snapshot to {image_path}")


The table above comes from `df.describe()` and is also saved as `artifacts/q1_describe_table.png`, which you can attach as the requested screenshot.
Only the numeric attributes (age, bmi, children, charges) appear because pandas defaults to summarizing numeric dtypes; categorical features such as `sex`, `smoker`, and `region` require `include="all"` to see their stats.
Notice how the `count` row is below 1,349 for every numeric column (e.g., bmi = 1,347), which confirms the missing instances mentioned in the previous subtask.


## c)

## d)

## e)

## f)

## g)

## h)

## i)

## j)